In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: 1
    Vertex: b2, Edge: 2

In [6]:
x = MechanismState{Float64}(mechanism);

In [7]:
rand!(x)
x.v

DataStructures.OrderedDict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  Joint "1": Quaternion f… => [0.5800426833926333,0.7150999217912095,0.78465196…
  Joint "2": Revolute joi… => [0.9091247344514048]

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
cache = MechanismStateCache(mechanism, x);

In [11]:
relative_transform(cache, joint2.frameAfter, body1.frame)

Transform3D from "after_2" to "body1":
rotation: 2.999916405772899 rad about [0.9310323375075998,-0.04731865785534665,0.3618559535697918], translation: Vec(0.20318229105230978,0.6366707084659566,0.6657360431301618)


In [12]:
center_of_mass(cache)

Point3D in "world": Vec(2.2171403490308483,0.45014096848793633,1.6058602248011975)

In [13]:
mass(mechanism)

0.74036036504049

In [14]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(1.189008484377002,0.8585463268571039,0.09822619504609323)

In [15]:
relative_twist(cache, body2, body1)

Twist of "after_2" w.r.t "after_1" in "world":
angular: Vec(-0.5534731190821882,-0.19117666451875165,0.7249188133585568), linear: Vec(0.6411549408158799,-0.9163001781155691,0.24787163727446565)

In [16]:
relative_twist(cache, body1, world) + relative_twist(cache, body2, body1) + relative_twist(cache, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [17]:
geometric_jacobian(cache, path(mechanism, body2, world))

GeometricJacobian: body: "world", base: "after_2", expressed in "world":
6x7 Array{Float64,2}:
  0.608798  -0.757152  -0.235601  -0.609273   -0.0       -0.0       -0.0     
  0.210287  -0.523799  -0.338341   0.781767   -0.0       -0.0       -0.0     
 -0.797381   0.390327  -0.911053  -0.132768   -0.0       -0.0       -0.0     
 -0.705244   0.166816  -0.43889   -0.0375894  -0.757152  -0.235601  -0.609273
  1.00789   -0.570554   1.39527    0.221576   -0.523799  -0.338341   0.781767
 -0.272649  -0.442068  -0.404668   1.47718     0.390327  -0.911053  -0.132768

In [29]:
showall(mass_matrix(cache))

[-0.2903726639020933 -0.10304840588126288 -0.6506119570414497 0.36248129322319367 -0.14510161294572818 -0.165132369345565 1.5967220394450097
 0.33500930314776983 0.16819738867359532 1.1839126237208253 -0.2915129294943822 0.6804914432984182 -0.11489666168221893 1.7425556059855563
 0.5826341605553947 0.008027956871968441 0.34731448911471247 -0.10635118791462966 -0.036832564806965104 0.7395338327026063 -4.4980513789230105
 0.7787118050385747 0.36244259511553467 0.8306878638987872 6.938893903907228e-18 -0.07532461638796542 0.20646783563830595 -3.396783947834983
 -0.07278931924388937 0.12927274705095249 0.04170178076829463 0.07532461638796542 0.0 -0.2967407615976061 1.3321858791163181
 0.5109904099506535 0.261980815609549 1.1753622322281165 -0.20646783563830595 0.29674076159760615 0.0 -0.6360373702291596
 1.5967220394450097 1.7425556059855563 -4.4980513789230105 -3.396783947834983 1.3321858791163181 -0.6360373702291596 -0.358541253178548]

In [19]:
A = momentum_matrix(cache)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
  0.0714151  -0.0273816  -0.0781603  …   0.051739    0.133799     0.0014952
 -0.0615988  -0.0349701  -0.0564072      0.0743012  -0.17168      1.2032   
  0.0558677   0.020068    0.0265402      0.200071    0.0291565    0.318452 
  0.883786    0.464499    1.3549         0.123764    0.0864151   -2.00276  
 -0.0162142   0.0287776  -0.469636      -0.271437    0.00774815  -0.837365 
 -0.302424    0.0110859  -0.130197   …   0.0687989  -0.350937     2.16576  

In [20]:
A * velocity_vector(x)

Momentum expressed in "world":
angular: Vec(0.0732113720666739,0.8950027951761239,0.39616654156137343), linear: Vec(0.15193686009636842,-1.1336777684494157,1.4784935342471766)

In [21]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(cache, accel, world.frame)

SpatialAcceleration of "after_2" w.r.t "world" in "world":
angular: Vec(0.2130463732019949,-0.012742862107630204,0.9985306298688836), linear: Vec(1.0219080166023022,-1.6825083416949664,1.3056382018167283)

In [22]:
bias_acceleration(cache, body2)

SpatialAcceleration of "after_2" w.r.t "world" in "world":
angular: Vec(0.0521485441182436,-1.0799964691815598,-0.24500302934418716), linear: Vec(-0.5342900666439648,-0.23510293705214474,-1.7973989814664317)

In [23]:
body2.inertia

SpatialInertia expressed in "b2":
mass: 0.5207557973418298
center of mass: Vec(0.6234800140340824,0.7260160004737968,0.8597582577125844)
moment of inertia:
FixedSizeArrays.Mat{3,3,Float64}(
    0.6055125593768358 0.35191203136633176 0.7261087435013003
    0.3998324077471602 0.3644042970645538 0.19315300451201156
    0.6153809242440376 0.8394583281482726 0.6733075044077661
)
